In [1]:
import os

In [2]:
%pwd

'c:\\Users\\jhasu\\OneDrive\\Desktop\\chickenclassifier\\chickendisease_classification\\research'

In [3]:
os.chdir("../")

In [13]:
%pwd

'c:\\Users\\jhasu\\OneDrive\\Desktop\\chickenclassifier\\chickendisease_classification'

In [14]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainingConfig:
    root_dir:Path
    trained_model_path:Path
    updated_base_model_path:Path
    training_data:Path
    params_epochs:int
    params_is_augmentation:bool
    params_image_size: list
    params_batch_size:int

@dataclass(frozen=True)
class CallBackConfig:
    root_dir:Path
    tensorboard_root_log_dir:Path
    checkpoint_model_filepath:Path


In [15]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml,create_directory
import tensorflow as tf

In [16]:
class ConfigManager:
    def __init__(self,
     config_filepath=CONFIG_FILE_PATH,
     param_filepath=PARAM_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.param = read_yaml(param_filepath)
        
    def get_callback_config(self)->CallBackConfig:
        config=self.config.prepare_callback
        model_ckpt_dir= os.path.dirname(config.checkpoint_model_filepath)
        create_directory([model_ckpt_dir])

        callback_config= CallBackConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )
        return callback_config
    
    def get_training_config(self)->ModelTrainingConfig:
        training=self.config.training
        params= self.param
        prepare_base_model= self.config.prepare_base_model

        training_data= os.path.join(self.config.data_ingestion.unzip_zar,"Chicken-fecal-images")

        create_directory([Path(training.root_dir)])

        model_trainig_config= ModelTrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(self.config.prepare_base_model.update_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE,
            params_batch_size=params.BATCH_SIZE
        )
        return model_trainig_config
    

In [17]:
import time

In [18]:
class preparecallback:
    def __init__(self,config:CallBackConfig):
        self.config=config

    @property
    def _create_tb_callbacks(self):
        timestamp= time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_ruuning_log_dir= os.path.join(
            self.config.tensorboard_root_log_dir, 
            f"ruuning_logs_at_{timestamp}",)
        return tf.keras.callbacks.TensorBoard(log_dir=tb_ruuning_log_dir)
    
    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only= True
        )

    def get_tb_ckpt_callback(self):
        return[
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [10]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [19]:
class Training:
    def __init__(self,config:ModelTrainingConfig):
        self.config=config

    def get_base_model(self):
        self.model= tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_genrator(self):
        datagenerator_kwargs= dict(
            rescale=1/255,
            validation_split=0.2,
        )
        dataflow_kwargs= dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinier")
        
        valid_datagenerator= tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        
        self.valid_genrator= valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator= tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                horizontal_flip=True,
                **datagenerator_kwargs
            )
            
        else:
            train_datagenerator= valid_datagenerator
        
        self.train_genrator= train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs)

    @staticmethod
    def save_model(path:Path, model: tf.keras.Model):
        model.save(path)
    
    def train(self,callback_list:list):
        self.step_per_epochs= self.train_genrator.samples // self.train_genrator.batch_size
        self.validation_steps= self.valid_genrator.samples // self.valid_genrator.batch_size
        self.model.fit(
            self.train_genrator,
            steps_per_epoch=self.step_per_epochs,
            epochs=self.config.params_epochs,
            validation_data=self.valid_genrator,
            validation_steps=self.validation_steps,
            callbacks=callback_list,
        )
        self.save_model(
            path=self.config.trained_model_path,
            model= self.model)


In [20]:
try:
    config=ConfigManager()
    prepare_callBacks_config= config.get_callback_config()
    prepare_Callbacks= preparecallback(config=prepare_callBacks_config)
    callback_list= prepare_Callbacks.get_tb_ckpt_callback()

    training_Config= config.get_training_config()
    training= Training(config= training_Config)
    training.get_base_model()
    training.train_valid_genrator()
    training.train(callback_list= callback_list)



except Exception as e:
    raise e

[2025-01-16 02:03:02,066: INFO : common : yaml file: config\config.yaml loaded successfully ]
[2025-01-16 02:03:02,073: INFO : common : yaml file: params.yaml loaded successfully ]
[2025-01-16 02:03:02,076: INFO : common : Directory: artifacts/prepare_callback/checkpoint_dir created successfully ]


[2025-01-16 02:03:02,110: INFO : common : Directory: artifacts\training created successfully ]
Found 78 images belonging to 2 classes.
Found 312 images belonging to 2 classes.
19/19 ━━━━━━━━━━━━━━━━━━━━ 35s 2s/step - accuracy: 0.5209 - loss: 10.1805 - val_accuracy: 0.6094 - val_loss: 25.9023
